In [19]:
import numpy as np

# Paso 1 ─────────────── Crear activaciones sintéticas (B × C × H × W)
rng = np.random.default_rng(0)
B, C, H, W = 3, 4, 5, 5          # 3 imágenes, 4 canales, 5×5 píxeles

acts_X = rng.standard_normal((B, C, H, W))
acts_Y = rng.standard_normal((B, C, H, W))

# Paso 2 ─────────────── Aplanar: (B × C × H × W) ➜ (B·H·W) × C
def flatten_conv(out):
    out = out.transpose(0, 2, 3, 1)      # B H W C
    return out.reshape(-1, out.shape[-1])  # (B·H·W) C

X = flatten_conv(acts_X)
Y = flatten_conv(acts_Y)

# Paso 3 ─────────────── Centrar
X_centered = X - X.mean(axis=0, keepdims=True)
Y_centered = Y - Y.mean(axis=0, keepdims=True)

# Paso 4 ─────────────── PCA clásica vía SVD, top‑k componentes
def topk_pca(mat, k):
    U, S, Vt = np.linalg.svd(mat, full_matrices=False)
    V = Vt.T                     # C × C  (eigenvectores de XᵀX)
    Vk = V[:, :k]                # C × k
    var_expl = (S**2) / (S**2).sum()
    return Vk, var_expl[:k]

k = 5
V_k, var_X = topk_pca(X_centered, k)
U_k, var_Y = topk_pca(Y_centered, k)

# Paso 5 ─────────────── Similaridad de subespacios
subspace_sim = (1 / k) * np.linalg.norm(V_k.T @ U_k, ord='fro')**2

# Mostrar resultados
print("Formas de las matrices después de aplanar:")
print("  X :", X.shape)
print("  Y :", Y.shape)
print("\nVarianza explicada (Task X) por los dos primeros PCs:")
print(" ", np.round(var_X, 4))
print("Varianza explicada (Task Y) por los dos primeros PCs:")
print(" ", np.round(var_Y, 4))
print(f"\nSubspaceSim_k (k={k}): {subspace_sim:.4f}")


Formas de las matrices después de aplanar:
  X : (75, 4)
  Y : (75, 4)

Varianza explicada (Task X) por los dos primeros PCs:
  [0.3342 0.2706 0.2169 0.1784]
Varianza explicada (Task Y) por los dos primeros PCs:
  [0.3003 0.2739 0.2285 0.1973]

SubspaceSim_k (k=5): 0.8000
